# Copy

In [1]:
function getMotif(DNA, ind, k, t)
    sample = Array{String,1}(undef, t)
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    return sample
end

getMotif (generic function with 1 method)

# Copy

In [2]:
function Score(DNA, t, k, ind)
    sample = Array{String,1}(undef, t)
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    score = 0
    for i=1:k
        count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
        for j=1:t
            count[sample[j][i]] = count[sample[j][i]] + 1
        end
        max = 0
        for each in count
            if each[2]>max
                max = each[2]
            end
        end
        score = score+max
    end
    return score/k
end

Score (generic function with 1 method)

# Copy

In [3]:
function genIndividual(t, ub)
    M = zeros(Int64, 1, t)
    s = rand(t)*(ub)
    for i in 1:t
        s[i] = s[i] + 1
        M[i] = trunc(Int64, s[i])
    end
    return M
end

genIndividual (generic function with 1 method)

# Copy

In [4]:
function employeeLookAround(ind, t, ub)
    i = trunc(Int64, rand() * t + 1)
    j = trunc(Int64, rand() * ub + 1)
    ind[i] = j
    return ind
end

employeeLookAround (generic function with 1 method)

# Copy

In [5]:
function crowdingDist(objs)
    crowdDist = zeros(Float64, size(objs)[2])
    for i in 1:size(objs)[1]
        v = sortperm(objs[i,:], rev=true)
        crowdDist[v[1]] = crowdDist[v[1]] + 2*objs[i,v[1]]
        crowdDist[v[end]] = crowdDist[v[end]] + 2*objs[i,v[end]]
        for j in 2:length(crowdDist)-1
            crowdDist[v[j]] = crowdDist[v[j]] + (objs[i,v[j-1]] + objs[i,v[j+1]])
        end
    end
    return crowdDist
end

crowdingDist (generic function with 1 method)

# Copy

In [6]:
function nds(metric)
    ndsArchive = Array{Int64,1}[]
    dominated = zeros(Int64, length(metric))
    dominates = Array{Int64}[]
    tempnds = []
    for i in 1:length(metric)
        tempDom = []
        for j in 1:length(metric)
            if metric[i] > metric[j]
                push!(tempDom, j)
            elseif metric[i] < metric[j]
                dominated[i] = dominated[i] + 1
            end
        end
        push!(dominates, tempDom)
        if dominated[i] == 0
            push!(tempnds, i)
        end
    end
    push!(ndsArchive, tempnds)
    i = 1
    while length(ndsArchive[i]) != 0
        tempndsnext = []
        tempndscurr = ndsArchive[i]
        for j in 1:length(tempndscurr)
            p = tempndscurr[j]
            currSet = dominates[p]
            for k in 1:length(currSet)
                q = currSet[k]
                dominated[q] = dominated[q] - 1
                if dominated[q] == 0
                    push!(tempndsnext, q)
                end
            end
        end
        push!(ndsArchive, tempndsnext)
        i = i + 1
    end
    return ndsArchive
end

nds (generic function with 1 method)

# Copy

In [7]:
function complexity(DNA, t, k, ind, prob)
    sample = Array{String,1}(undef, t)
    for j=1:t
        sample[j] = DNA[j][ind[j]:(ind[j]+k-1)]
    end
    den = 1
    consensusDict = zeros(Int64, 4, 1)
    for i=1:k
        count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
        for j=1:t
            count[sample[j][i]] = count[sample[j][i]] + 1
        end
        countN = []
        for each in count
            push!(countN, each[2])
        end
        max = 0    
        consensus = i
        for i in 1:4
            if countN[i] > max
                max = countN[i]
                consensus = i
            end
        end
        consensusDict[consensus] = consensusDict[consensus] + 1
    end
    consensusDict = consensusDict .+ 1
    consensusDict = consensusDict ./ (k+4)
    complexityScore = 0
    for i in 1:4
        temp = log(2, consensusDict[i]/prob[i]) * consensusDict[i]
        complexityScore = complexityScore + temp
    end
    return complexityScore
end

complexity (generic function with 1 method)

# Copy

In [8]:
function getNucProb(DNA)
    count = Dict('A'=>0,'C'=>0,'G'=>0,'T'=>0)
    n = length(DNA[1])
    t = length(DNA)
    for i in 1:t
        for j in 1:n
            count[DNA[i][j]] = count[DNA[i][j]] + 1
        end
    end
    countN = []
    for each in count
        push!(countN, each[2])
    end
    countSum = sum(countN)
    prob = countN ./ countSum
    return prob
end

getNucProb (generic function with 1 method)

# Copy

In [9]:
function extract(archive)
    temp = []
    for each in archive
        append!(temp, each)
    end
    result = Array{Int64, 1}(undef, length(temp))
    for i in 1:length(temp)
        result[i] = temp[i]
    end
    return result
end

extract (generic function with 1 method)

In [10]:
function moabc(DNA, t, N, MFE = 100)
    ndsArchiveExtract = collect(1:N+3)
    ndsArchive = []
    objs=[]
    prob = getNucProb(DNA)
    eval = 0
    n = length(DNA[1])
    Bees = zeros(Int64, N + 3, t)
    fitness = zeros(Float64, N + 3, 1)
    complexityScore = zeros(Float64, N + 3, 1)
    crowdDist = []
    k = zeros(Int64, N+3, 1)
    s = rand(trunc(Int64, N/2))*(64 - 7 + 1)
    for i in 1:trunc(Int64, N/2)
        s[i] = s[i] + 7
        k[i] = trunc(Int64, s[i])
    end
    for i in 1:trunc(Int64, N/2)
        ub = n - k[i] + 1
        temp = genIndividual(t,ub)
        Bees[i,:] = temp
        fitness[i] = Score(DNA, t, k[i], Bees[i,:])
        complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
    end
    best = 1
    while eval < MFE
        for i in 1:trunc(Int64, N/2)     #Employee
            ub = n - k[i] + 1
            temp = employeeLookAround(copy(Bees[i,:]), t, ub)
            tempScore = Score(DNA, t, k[i], temp)
            tempComplexity = complexity(DNA, t, k[i], temp, prob)
            if (tempScore > fitness[i])
                Bees[i,:] = temp
                fitness[i] = tempScore
                complexityScore[i] = tempComplexity
            end
        end
        fitnessSum = sum(fitness[1:trunc(Int64, N/2)])
        probArray = fitness[1:trunc(Int64, N/2)] ./ fitnessSum
        pdfArray = zeros(Float64, trunc(Int64, N/2), 1)
        for i in 1:trunc(Int64, N/2)
            pdfArray[i] = sum(probArray[1:i])
        end
        for i in trunc(Int64, N/2)+1:N      #onlooker phase
            p = rand()
            final = 1
            for i in 1:length(pdfArray)
                if (p < pdfArray[i])
                    final = i
                    break
                end
            end
            k[i] = k[final]
            ub = n - k[i] + 1
            temp = employeeLookAround(copy(Bees[final,:]), t, ub)
            tempScore = Score(DNA, t, k[i], temp)
            tempComplexity = complexity(DNA, t, k[i], temp, prob)
            if (tempScore > fitness[final])
                Bees[i,:] = temp
                fitness[i] = tempScore
                complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
            else
                Bees[i,:] = copy(Bees[final,:])
                fitness[i] = fitness[final]
                complexityScore[i] = complexityScore[final]
            end
        end
        for i in N+1:N+3        # scout phase
            k[i] = trunc(Int64, rand()*(64 - 7 + 1) + 7)
            ub = n - k[i] + 1
            temp = genIndividual(t,ub)
            Bees[i,:] = temp
            fitness[i] = Score(DNA, t, k[i], Bees[i,:])
            complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
        end
        objs = zeros(Float64, 3, N+3)
        objs[1,:] = fitness
        kTemp = k ./ 64
        objs[2,:] = kTemp
        objs[3,:] = complexityScore
        crowdDist = crowdingDist(objs)
        ndsArchive = nds(crowdDist)
        ndsArchiveExtract = extract(ndsArchive)
        k[:] = k[ndsArchiveExtract]
        fitness[:] = fitness[ndsArchiveExtract]
        complexityScore[:] = complexityScore[ndsArchiveExtract]
        #println(Bees)
        #println(crowdDist)
        #println(ndsArchive)
        #println(ndsArchiveExtract)
        for i in 1:size(Bees)[2]
            Bees[:,i] = Bees[:,i][ndsArchiveExtract]
        end
        #println(Bees)
        best = 1
        eval = eval + 1
    end
    println(k[best])
    println(fitness[best])
    println(complexityScore[best])
    println(Bees[best,:])
    answer = getMotif(DNA, Bees[best,:], k[best], t)
    return answer
end

moabc (generic function with 2 methods)

In [11]:
function abc(DNA, k, t, N, limit = 50, MFE = 100)
    evals = 0
    n = length(DNA[1])
    ub = n - k + 1
    Bees = zeros(Int64, N, t)
    fitness = zeros(Float64, N, 1)
    trials = zeros(Int64, N, 1)
    for i in 1:trunc(Int64, N)
        temp = genIndividual(t,ub)
        Bees[i,:] = temp
        fitness[i] = Score(DNA, t, k, Bees[i,:])
    end
    while evals < 200
        for i in 1:N #Employee looks around
            temp = employeeLookAround(copy(Bees[i,:]), t, ub)
            tempScore = Score(DNA, t, k, temp)
            if tempScore > fitness[i]
                Bees[i,:] = temp
                fitness[i] = tempScore
                trials[i] = 0
            else
                trials[i] = trials[i] + 1
            end
        end
        fitnessSum = sum(fitness)
        probArray = fitness ./ fitnessSum
        for i in 1:N #Onlooker tries to improve the best ones
            r = rand()
            temp = employeeLookAround(copy(Bees[i,:]), t, ub)
            tempScore = Score(DNA, t, k, temp)
            if (tempScore > fitness[i]) & (r < probArray[i])
                Bees[i,:] = temp
                fitness[i] = tempScore
                trials[i] = 0
            else
                trials[i] = trials[i] + 1
            end
        end
        for i in 1:N #Scouts try to replace overexploited source
            if trials[i] > limit
                temp = genIndividual(t, ub)
                tempScore = Score(DNA, t, k, temp)
                if tempScore > fitness[i]
                    Bees[i,:] = temp
                    fitness[i] = tempScore
                    trials[i] = 0
                else
                    trials[i] = trials[i] + 1
                end
            end
        end
        evals = evals + 1
    end
    fitMax = findmax(fitness)[1]
    bestSol = 1
    for i in 1:length(fitness)
        if fitMax == fitness[i]
            bestSol = i
        end
    end
    println(Score(DNA, t, k, Bees[bestSol,:]) * k)
    answer = getMotif(DNA, Bees[bestSol,:], k, t)
    return answer
end

abc (generic function with 3 methods)

In [38]:
function mmoabc(DNA, t, N, MFE = 100)
    ndsArchiveExtract = collect(1:N+3)
    ndsArchive = []
    prob = getNucProb(DNA)
    eval = 0
    n = length(DNA[1])
    Bees = zeros(Int64, N + 3, t)
    fitness = zeros(Float64, N + 3, 1)
    complexityScore = zeros(Float64, N + 3, 1)
    crowdDist = []
    k = zeros(Int64, N+3, 1)
    s = rand(trunc(Int64, N/2))*(64 - 7 + 1)
    for i in 1:trunc(Int64, N/2)
        s[i] = s[i] + 7
        k[i] = trunc(Int64, s[i])
    end
    for i in 1:trunc(Int64, N/2)
        ub = n - k[i] + 1
        temp = genIndividual(t,ub)
        Bees[i,:] = temp
        fitness[i] = Score(DNA, t, k[i], Bees[i,:])
        complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
    end
    best = 1
    while eval < MFE
        for i in 1:trunc(Int64, N/2)     #Employee
            ub = n - k[i] + 1
            temp = employeeLookAround(copy(Bees[i,:]), t, ub)
            tempScore = Score(DNA, t, k[i], temp)
            tempComplexity = complexity(DNA, t, k[i], temp, prob)
            if (tempScore > fitness[i])
                Bees[i,:] = temp
                fitness[i] = tempScore
                complexityScore[i] = tempComplexity
            end
        end
        fitnessSum = sum(fitness[1:trunc(Int64, N/2)])
        probArray = fitness[1:trunc(Int64, N/2)] ./ fitnessSum
        pdfArray = zeros(Float64, trunc(Int64, N/2), 1)
        for i in 1:trunc(Int64, N/2)
            pdfArray[i] = sum(probArray[1:i])
        end
        for i in trunc(Int64, N/2)+1:N      #onlooker phase
            p = rand()
            final = 1
            for i in 1:length(pdfArray)
                if (p < pdfArray[i])
                    final = i
                    break
                end
            end
            k[i] = k[final]
            ub = n - k[i] + 1
            temp = employeeLookAround(copy(Bees[final,:]), t, ub)
            tempScore = Score(DNA, t, k[i], temp)
            tempComplexity = complexity(DNA, t, k[i], temp, prob)
            if (tempScore > fitness[final])
                Bees[i,:] = temp
                fitness[i] = tempScore
                complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
            else
                Bees[i,:] = copy(Bees[final,:])
                fitness[i] = fitness[final]
                complexityScore[i] = complexityScore[final]
            end
        end
        maxFitness = argmax(fitness).I[1]
        for i in N+1:N+3        # scout phase
            kTemp= trunc(Int64, rand()*(64 - 7 + 1) + 7)
            ub = n - kTemp + 1
            temp = genIndividual(t,ub)
            tempScore = Score(DNA, t, kTemp, temp)
            if tempScore > fitness[maxFitness]
                Bees[i,:] = temp
                k[i] = kTemp
            else
                Bees[i,:] = Bees[maxFitness,:]
                k[i] = k[maxFitness]
            end
            fitness[i] = Score(DNA, t, k[i], Bees[i,:])
            complexityScore[i] = complexity(DNA, t, k[i], Bees[i,:], prob)
        end
        
        objs = zeros(Float64, 3, N+3)
        objs[1,:] = fitness
        kTemp = k ./ 64
        objs[2,:] = kTemp
        objs[3,:] = complexityScore
        crowdDist = crowdingDist(objs)
        ndsArchive = nds(crowdDist)
        ndsArchiveExtract = extract(ndsArchive)
        k[:] = k[ndsArchiveExtract]
        fitness[:] = fitness[ndsArchiveExtract]
        complexityScore[:] = complexityScore[ndsArchiveExtract]
        #println(Bees)
        #println(crowdDist)
        #println(ndsArchive)
        #println(ndsArchiveExtract)
        for i in 1:size(Bees)[2]
            Bees[:,i] = Bees[:,i][ndsArchiveExtract]
        end
        #println(Bees)
        best = 1
        eval = eval + 1
    end
    println(k[best])
    println(fitness[best])
    println(complexityScore[best])
    println(Bees[best,:])
    answer = getMotif(DNA, Bees[best,:], k[best], t)
    return answer
end

mmoabc (generic function with 2 methods)

In [57]:
open("test1.txt") do file
    x = readline(file)
    line1 = split(x," ")
    t = parse(Int64, line1[1])
    DNA = Array{String,1}(undef,t)
    for i in 1:t
        DNA[i] = readline(file)
    end
    println(length(DNA[1]))
    #@time Motif1 = abc(DNA, k, t, N, 50, 500)
    #for each in Motif1
    #    println(each)
    #end
    println(t*3/4)
    @time Motif2 = moabc(DNA, t, 200, t*50)
    for each in Motif2
        println(each)
    end
    @time Motif3 = mmoabc(DNA, t, 200, t*50)
    for each in Motif3
        println(each)
    end
end

200
3.75
10
4.5
0.2616957797592377
[151, 108, 33, 7, 32]
  0.645622 seconds (7.26 M allocations: 819.220 MiB, 14.91% gc time)
GGCGGGGGGG
GGCGGCGAGG
GGCGGGGGGC
CCCGGGGGGG
GGCGGGGGGG
63
4.619047619047619
0.025672227987114278
[55, 102, 65, 32, 61]
  2.762951 seconds (33.14 M allocations: 3.754 GiB, 14.72% gc time)
CGCCAGGGCGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCGGGGCGGGAC
CGCCAGGGCGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGCGCTCCCGCCCCCC
CGCCAGGGCGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGCGCGCCCGTAAGGG
CGCCAGGGCGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCCCCAAGTCCT
CGCCAGGGCGGCGAGGGCGCGTCGGGCCGGGCGCAGGGCCGGCGGGCCGCGCCTCGGCCTGGG
